In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from tqdm.notebook import tqdm_notebook
from concurrent.futures import ThreadPoolExecutor


In [2]:
session = requests.Session()

payload = {
    'userid': '22BCE2454',
    'password': '#Un3599known'
}

url = "http://webopac.vit.ac.in/"
s = session.post(url, data=payload)
s

<Response [200]>

In [3]:
final_df = pd.DataFrame()

In [4]:
def scrape_page(i):
    webpage = session.get(f'http://webopac.vit.ac.in/cgi-bin/koha/opac-search.pl?&limit=yr%2Cst-numeric%3D1984-&offset={i}&sort_by=relevance').text
    soup = BeautifulSoup(webpage, 'lxml')
    content = soup.find_all('td', class_='bibliocol')
    
    title = []
    author = []
    barcode = []
    location = []
    material_type = []
    format_form = []
    literary_form = []
    publisher = []
    available = []
    img_link = []

    for j in content:
        try:
            title.append(j.find('a', class_='title').text.strip())
        except:
            title.append(np.nan)
        try:
            author.append(j.find('span', class_='author').text.strip())
        except:
            author.append(np.nan)
        try:
            barcode.append(j.find('span', class_='results_summary Barcode').contents[-1].strip())
        except:
            barcode.append(np.nan)
        try:
            location.append(j.find('span', class_='results_summary Location').contents[-1].strip())
        except:
            location.append(np.nan)
        try:
            material_type.append(j.find('span', class_='results_material_type').contents[-1].strip())
        except:
            material_type.append(np.nan)
        try:
            format_form.append(j.find('span', class_='results_format').contents[-1].strip())
        except:
            format_form.append(np.nan)
        try:
            literary_form.append(j.find('span', class_='results_contents_literary').contents[-1].strip())
        except:
            literary_form.append(np.nan)
        try:
            publisher.append(j.find('span', class_='results_summary publisher').contents[-1].strip())
        except:
            publisher.append(np.nan)
        try:
            available.append(j.find('span', class_='available').text.strip())
        except:
            available.append(np.nan)
        try:
            img_link.append(j.find('img').get('src'))
        except:
            img_link.append(np.nan)

    df = pd.DataFrame({
        'Title': title,
        'Author': author,
        "Barcode Number": barcode,
        "Location In Library": location,
        "Material type": material_type,
        "Format": format_form,
        "Literary form": literary_form,
        "Publisher": publisher,
        "Availability": available,
        "Image": img_link
    })

    return df

In [5]:
# Define the range of offsets to scrape
offsets = range(0, 92940, 20)

# Use ThreadPoolExecutor to scrape pages in parallel
with ThreadPoolExecutor(max_workers=8) as executor:
    dataframes = list(tqdm_notebook(executor.map(scrape_page, offsets), total=len(offsets)))

  0%|          | 0/4647 [00:00<?, ?it/s]

In [6]:
# Concatenate the dataframes
final_df = pd.concat(dataframes, ignore_index=True)
final_df

,Title,Author,Barcode Number,Location In Library,Material type,Format,Literary form,Publisher,Availability,Image
0,changing places,judy kramer.,116002.,IIIF-R1-C4-C,Text,print,Not fiction,new york ; riverhead books ; 2000,Items available for loan: Vellore Institute of...,https://images-na.ssl-images-amazon.com/images...
1,chapel noir: an irene adler novel,carole nelson douglas.,116004.,IIF-R70-C1-E,Text,print,Not fiction,new york ; a forge book ; 2001,Items available for loan: Vellore Institute of...,https://images-na.ssl-images-amazon.com/images...
2,coast road,barbara delinsky.,116023.,IIF-R70-C4-B,Text,print,Not fiction,simon ANDamp; schuster ; 1998,Items available for loan: Vellore Institute of...,https://images-na.ssl-images-amazon.com/images...
3,collision bend,roberts les.,116033.,IIF-R72-C3-B,Text,print,Not fiction,new york ; st. martin`s press inc. ; 1996,Items available for loan: Vellore Institute of...,https://images-na.ssl-images-amazon.com/images...
4,compat,coonts stephen.,116037.,IIF-R70-C2-F,Text,print,Not fiction,new york ; a tom doherty associates book ; ...,Items available for loan: Vellore Institute of...,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...,...,...,...,...
90199,FINANCIAL VALUATION AND ECONOMETRICS,LIM KIAN GUAN.,245666.,,Text,print,Not fiction,LONDON ; WORLD SCIENTIFIC ; 2020,Items available for reference: Vellore Institu...,https://images-na.ssl-images-amazon.com/images...
90200,INTRODUCTION TO MICROFINANCE,WATKINS TODD A.,245665.,,Text,print,Not fiction,LONDON ; WORLD SCIENTIFIC ; 2022,Items available for reference: Vellore Institu...,https://images-na.ssl-images-amazon.com/images...
90201,R IN FINANCE AND ECONOMICS A BEGINNER GUIDE,ABHAY KUMAR SINGH ; DAVID EDMUND ALLEN.,245664.,,Text,print,Not fiction,LONDON ; WORLD SCIENTIFIC ; 2021,Items available for reference: Vellore Institu...,/opac-tmpl/lib/famfamfam/BK.png
90202,FOUNDATIONS OF QUANTUM FIELD THEORY,ROTHE KLAUS D.,245663.,,Text,print,Not fiction,LONDON ; WORLD SCIENTIFIC ; 2021,Items available for reference: Vellore Institu...,https://images-na.ssl-images-amazon.com/images...


In [23]:
final_df.duplicated().sample(10)

21695    False
30833    False
31990    False
67838    False
51087     True
53805    False
52734    False
69014    False
44246     True
59469     True
dtype: bool

In [17]:
final_df.drop(final_df.duplicated(),axis='columns',inplace=True)

KeyError: '[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, F

In [7]:
final_df.to_csv('NewKoha.csv')